In [1]:
import pandas as pd 
import numpy as np 

from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import StackingRegressor

from sklearn.impute import SimpleImputer

import warnings
warnings.simplefilter('ignore')

In [3]:
%%time

df = pd.read_csv("SUPCOM_Train.csv")
df.dropna(thresh=df.shape[0]*0.3,how='all',axis=1, inplace = True) #remove de variables with more than 30% of NA values

X = df.drop(['target', 'id'], 1)
y = df['target']

my_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

X = X[my_cols]

imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X)

teste = pd.read_csv("SUPCOM_Test.csv")
teste = teste[my_cols]

imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X)

aux = imputer.transform(X)
X_pp = pd.DataFrame(aux, columns = X.columns)

aux = imputer.transform(teste)
teste_pp = pd.DataFrame(aux, columns = teste.columns)

# estimators = [('lasso', Lasso(alpha = 0.0001)),
              
#               ('rf', RandomForestRegressor(n_estimators=400, min_samples_split=2, 
#                                            min_samples_leaf=1, max_features='sqrt', 
#                                            max_depth=None, bootstrap=False)),
              
#               ('xgb', xgb.XGBRegressor(min_child_weight = 1, max_depth = 15, 
#                                        learning_rate = 0.05, gamma = 0.3, 
#                                        colsample_bytree = 0.7)),
              
#               ('lgbm', lgb.LGBMRegressor(num_leaves = 2620, min_split_gain = 0.15, 
#                                          max_depth = 15, learning_rate = 0.1, 
#                                          lambda_l2 = 20, lambda_l1 = 10, 
#                                          colsample_bytree = 0.5, boosting_type = 'gbdt')),
              
#               ('knn', KNeighborsRegressor(weights = 'distance', n_neighbors = 33, 
#                                           algorithm = 'auto'))]

estimators = [('rf', RandomForestRegressor(n_estimators = 200,
                                         min_samples_split = 6,
                                         min_impurity_decrease = 0.0,
                                         max_features = 'sqrt',
                                         max_depth = 25,
                                         criterion = 'mae',
                                         bootstrap = True,
                                         random_state = 0)),
              ('xgb', xgb.XGBRegressor(tree_method = 'exact',
                                     objective = 'reg:squarederror',
                                     n_estimators = 1600,
                                     min_child_weight = 6,
                                     max_depth = 8,
                                     gamma = 0,
                                     eta = 0.1,
                                     random_state = 0))]

model = StackingRegressor(estimators = estimators, final_estimator = GradientBoostingRegressor(random_state=0))
model.fit(X_pp, y)

p = model.predict(teste_pp)

teste = pd.read_csv("SUPCOM_Test.csv")

d = {'id': teste['id'], 'target': p} 

predictions = pd.DataFrame(d)
predictions.to_csv("submission.csv", index=False)

Wall time: 1h 24min 25s


Zindi Result: 5.517257859158874